In [4]:
import cv2 as cv
import numpy as np

KNOWN_DISTANCE = 11.43
VEHICLE_WIDTH = 70

CONFIDENCE_THRESHOLD = 0.2
NMS_THRESHOLD = 0.1

HIGHLIGHT_COLOR = (0, 255, 0)
DISTANCE_BOX_COLOR = (0, 0, 0)
ALERT_COLOR = (0, 0, 255)

yoloNet = cv.dnn.readNet('darknetobj.weights', 'darknetobj.cfg')

yoloNet.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
yoloNet.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA_FP16)

model = cv.dnn_DetectionModel(yoloNet)
model.setInputParams(size=(416, 416), scale=1/255, swapRB=True)
#change to video path to for webcam real time data
#cap = cv.VideoCapture(0)

cap = cv.VideoCapture("dashcam.mp4")

while True:
    ret, frame = cap.read()
    
    if not ret:
        break

    classes, scores, boxes = model.detect(frame, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)

    if classes is not None:
        for i in range(len(classes)):
            class_id = int(classes[i])
            score = float(scores[i])
            box = boxes[i]
                        
            if class_id in [0, 1, 2]: 
                distance = (VEHICLE_WIDTH * KNOWN_DISTANCE) / box[2]

                object_color = HIGHLIGHT_COLOR
                text_color = (0, 0, 0)  

                if distance < 5: 
                    object_color = ALERT_COLOR
                    text_color = (0, 0, 255)  

                cv.rectangle(frame, box, object_color, 2)  

                distance_text = f"{round(distance, 2)} m"
                text_size, _ = cv.getTextSize(distance_text, cv.FONT_HERSHEY_SIMPLEX, 0.7, 2)  
                text_x = box[0] + int((box[2] - text_size[0]) / 2)
                text_y = box[1] - 10
                cv.rectangle(frame, (text_x - 2, text_y - text_size[1] - 2),
                             (text_x + text_size[0] + 2, text_y + 2), DISTANCE_BOX_COLOR, -1)
                cv.putText(frame, distance_text, (text_x, text_y),
                           cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2) 

                if distance < 3:
                    alert_text = "Break"
                    alert_size, _ = cv.getTextSize(alert_text, cv.FONT_HERSHEY_SIMPLEX, 0.7, 2)  
                    alert_x = box[0] + int((box[2] - alert_size[0]) / 2)
                    alert_y = box[1] - 30  
                    cv.rectangle(frame, (alert_x - 2, alert_y - alert_size[1] - 2),
                                 (alert_x + alert_size[0] + 2, alert_y + 2), DISTANCE_BOX_COLOR, -1)
                    cv.putText(frame, alert_text, (alert_x, alert_y),
                               cv.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2, cv.LINE_AA)  # Changed thickness here

    cv.imshow('Road Object Detection', frame)
    
    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()
